In [1]:
import pandas as pd
from educonvokit import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
UPTAKE_HF_MODEL_NAME = 'ddemszky/uptake-model'
UPTAKE_MIN_NUM_WORDS_SPEAKER_A = 3
HIGH_UPTAKE_THRESHOLD = 0.5
UPTAKE_MAX_INPUT_LENGTH = 120

In [8]:
data = pd.read_excel('data/with_all_uptakes_new3.xlsx')
data.head()

,id,conversation_id,text,seq.type,student.cefr.level,student.L1,participant,label,human_int,text0,...,LCS,common_words,first_role,perc_dial1,perc_dial2,perTinS,perSinT,text_proc,prev_text_proc,cos_proc_pages
0,45939,7,"TEACHER:\n Hi there <STUDENT>, all OK?\n\nS...",['opening'],C1,Spanish,5d2b153afa24ba00173e8ee0,"['EXP INT 1', 'INT 1']",1,"TEACHER:\n Hi there <STUDENT>, all OK?\n===...",...,1,1,teacher,0.333333,0.500000,0.500000,0.333333,hi student ok hi teacher you,NaN,0.000000
1,22314,118,TEACHER:\n Hi - welcome to the chat\n Ho...,['topic opening'],C2,Italian,5ef3efcb420ab11a45a0b8a6,"['EXP INT 2', 'INT 2']",2,TEACHER:\n Hi - welcome to the chat\n Ho...,...,1,0,teacher,0.000000,0.000000,0.000000,0.000000,hi welcome chat you world thank you thank lett...,hello student hi teacher,0.668713
2,22315,118,"TEACHER:\n Oh Austria, interesting!\n Al...",['topic development'],C2,Italian,5ef3efcb420ab11a45a0b8a6,"['EXP INT 1', 'INT 1']",1,"TEACHER:\n Oh Austria, interesting!\n Al...",...,0,0,teacher,0.000000,0.000000,0.000000,0.000000,oh austria interesting pastry music yes nice p...,hi welcome chat you world thank you thank lett...,0.710058
3,22316,118,"TEACHER:\n I'm from Ukraine, but I've lived...",[],C2,Italian,5ef3efcb420ab11a45a0b8a6,"['EXP INT 2', 'INT 3']",3,"TEACHER:\n I'm from Ukraine, but I've lived...",...,2,3,teacher,0.300000,0.125000,0.125000,0.300000,i m ukraine i ve lived uk last years speak ger...,oh austria interesting pastry music yes nice p...,0.571614
4,22317,118,"TEACHER:\n Yes, I was an English teacher an...",[],C2,Italian,5ef3efcb420ab11a45a0b8a6,"['EXP INT 3', 'INT 3']",3,"TEACHER:\n Yes, I was an English teacher an...",...,1,3,teacher,0.074074,0.111111,0.111111,0.074074,yes english teacher cambridge examiner back uk...,i m ukraine i ve lived uk last years speak ger...,0.779869


In [10]:
conv_data = data[['id', 'first_role','conversation_id', 'text',  'dial1', 'dial2', 'Flag']].drop_duplicates().reset_index(drop=True)


conv_data['prev_text'] = conv_data.sort_values(['id']).groupby(['conversation_id', "Flag"])['text'].shift(1)
conv_data['prev_text'] = conv_data['prev_text'].fillna("")
conv_data['second_role'] =  conv_data['first_role'].apply(get_second_speaker)

conv_data = conv_data.sort_values(['id']).reset_index(drop=True)
conv_data

,id,first_role,conversation_id,text,dial1,dial2,Flag,prev_text,second_role
0,22313,teacher,118,TEACHER:\n Hello <STUDENT>!\n\nSTUDENT:\n ...,TEACHER:\n Hello <STUDENT>!,STUDENT:\n Hi <TEACHER>!,0,,student
1,22314,teacher,118,TEACHER:\n Hi - welcome to the chat\n Ho...,TEACHER:\n Hi - welcome to the chat\n Ho...,STUDENT:\n Thank you! And thank you for let...,0,TEACHER:\n Hello <STUDENT>!\n\nSTUDENT:\n ...,student
2,22315,teacher,118,"TEACHER:\n Oh Austria, interesting!\n Al...","TEACHER:\n Oh Austria, interesting!\n Al...","STUDENT:\n Yes, it is a nice place to be :)...",0,TEACHER:\n Hi - welcome to the chat\n Ho...,student
3,22316,teacher,118,"TEACHER:\n I'm from Ukraine, but I've lived...","TEACHER:\n I'm from Ukraine, but I've lived...","STUDENT:\n Oh, I see...so you also know wha...",0,"TEACHER:\n Oh Austria, interesting!\n Al...",student
4,22317,teacher,118,"TEACHER:\n Yes, I was an English teacher an...","TEACHER:\n Yes, I was an English teacher an...",STUDENT:\n Sort of polyglot...but I enjoy l...,0,"TEACHER:\n I'm from Ukraine, but I've lived...",student
...,...,...,...,...,...,...,...,...,...
6651,67110,student,52,STUDENT:\n let me think\n past simple!\n...,STUDENT:\n let me think\n past simple!,TEACHER:\n OK so remember the result will b...,1,STUDENT:\n because everyday and every time ...,teacher
6652,67111,student,52,STUDENT:\n because in recent year is presen...,STUDENT:\n because in recent year is presen...,TEACHER:\n YEs good! that keyword is 'recen...,1,STUDENT:\n let me think\n past simple!\n...,teacher
6653,67112,student,52,STUDENT:\n yes.. I think I can understand i...,STUDENT:\n yes.. I think I can understand i...,TEACHER:\n Yes I see what you mean about 'i...,1,STUDENT:\n because in recent year is presen...,teacher
6654,67113,student,52,STUDENT:\n yes that is what I like to do\n\...,STUDENT:\n yes that is what I like to do,TEACHER:\n OK great thanks <STUDENT> see yo...,1,STUDENT:\n yes.. I think I can understand i...,teacher


In [11]:
# # Apply the function to each group

# conv_data = conv_data.dropna(subset=['first_role', 'dial1', 'second_role', 'dial2'])

result = conv_data.groupby(['conversation_id', 'Flag']).apply(get_df_for_convid).reset_index(drop=True)



In [16]:
result = conv_data.groupby(['conversation_id', 'Flag']).apply(get_df_for_convid).reset_index()

In [17]:
pivoted_df = result.groupby('id').agg({
    'text': ' '.join,  # Join texts with a space
    'teacher_uptake_student': lambda x: x.dropna().max() if not x.dropna().empty else None,  # Keep only non-NaN
    'student_uptake_teacher': lambda x: x.dropna().max() if not x.dropna().empty else None   # Keep only non-NaN
}).reset_index()
pivoted_df

,id,text,teacher_uptake_student,student_uptake_teacher
0,22313,TEACHER:\n Hello <STUDENT>! STUDENT:\n H...,NaN,0.999087
1,22314,TEACHER:\n Hi - welcome to the chat\n Ho...,0.998000,0.938766
2,22315,"TEACHER:\n Oh Austria, interesting!\n Al...",0.986824,0.627085
3,22316,"TEACHER:\n I'm from Ukraine, but I've lived...",0.997537,0.996571
4,22317,"TEACHER:\n Yes, I was an English teacher an...",0.991618,0.999184
...,...,...,...,...
6651,67110,STUDENT:\n let me think\n past simple! T...,0.693843,0.997308
6652,67111,STUDENT:\n because in recent year is presen...,0.998417,0.951334
6653,67112,STUDENT:\n yes.. I think I can understand i...,0.276067,0.774874
6654,67113,STUDENT:\n yes that is what I like to do TE...,0.747850,0.204560
